In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY') 

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain

from PyPDF2 import PdfReader
# import google.generativeai as genai

In [8]:
# TODO:
# 1. store doc embeddings 
# 2. check google embeddings on larger corpus
# 3. github instructions
# 4. py script

yes


In [35]:
def get_embedds(file_path, filename):

    if os.path.exists('doc_embedddings'):
        print('searching for doc embedding')

    else:
        os.mkdir('doc_embedddings')


    reader = PdfReader(file_path)
    corpus = ''.join([p.extract_text() for p in reader.pages if p.extract_text()])

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_text(corpus)

    if len(chunks) >=50:
        chunks = chunks[:50]

    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    # vectors = embeddings.embed_documents(chunks)
    vectors = FAISS.from_texts(chunks, embeddings)

    return vectors


In [36]:
files = os.listdir('./pdfs/')
file = './pdfs/' + files[0]
print('filename: ', str(file))

filename:  ./pdfs/A Deep Neural Network for Unsupervised Anomaly Detection and Diagnosis in Multivariate Time Series Data.pdf


In [37]:
vectors = get_embedds(file)

In [4]:
# with open('dd.txt', 'r', encoding='utf-8') as f:
#     corpus = f.readlines()

# ###### CHANGE THIS ######
# corpus2 = corpus[:100]
# corpus2 = ' '.join(corpus)
# len(corpus2)

42632

In [38]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectors.as_retriever(), return_source_documents=True)

In [39]:
query = 'tell me about nsupervised Anomaly Detection'

In [40]:
chat_history = []
result = qa({"question": query, "chat_history": chat_history})
chat_history.append((query, result["answer"]))

In [42]:
print(chat_history[-1][1])

Unsupervised anomaly detection is a technique used to identify unusual patterns or data points in a dataset without relying on labeled examples of what constitutes an anomaly. This is particularly useful when you don't have a predefined set of "normal" or "abnormal" data points to train a model on.

Here's a breakdown of the key aspects of unsupervised anomaly detection:

**How it Works:**

* **No Labeled Data:** Unlike supervised methods, unsupervised anomaly detection doesn't require you to have a dataset where anomalies are explicitly marked.
* **Finding Outliers:** The goal is to identify data points that deviate significantly from the "normal" behavior or distribution of the data. These deviations are considered anomalies.
* **Assumptions:**  Unsupervised methods often rely on assumptions about the data, such as the expectation that most data points should follow a certain pattern or distribution.

**Common Techniques:**

1. **Distance-Based Methods:**
   * **k-Nearest Neighbors (